# 1. LLaMA-Factory `Evaluation` Tab

## 1.1 Understand Evaluation Output
[CSDN blog](https://blog.csdn.net/weixin_45573296/article/details/141333719)
- BLEU-4: BLEU分数：评价一种语言翻译成另一种语言的文本质量的指标。BLEU分数越高，表示翻译质量越高。
- ROUGE: 指标是在机器翻译、自动摘要、问答生成等领域常见的评估指标。
ROUGE+通过将模型生成的摘要或者回答与参考答案(一般是人工生成的)进行比较计算，得到对应的得分。
ROUGE指标与BLEU指标非常类似，均可用来衡量生成结果和标准结果的匹配程度，不同的是ROUGE基于`召回率`，BLEU更看重`准确率`。
- ROUGE也分为四种方法:
    - ROUGE-N (N=1,2,3,4)
    - ROUGE-L
    - ROUGE-W
    - ROUGE-S



<img src="./assets/Evaluation-Output.jpg" style="margin-left: 0px" width=1024px>

# 2. LLaMA-Factory QLoRA: Quantization at training stage

## 2.1 What is the problem to solve by this advanced LoRA?

### If we are using a 4090 GPU with 24GB memory to train a 1B model with larger batch size:

<img src="./assets/Out-Of-Memory.jpg" style="margin-left: 0px" width=1024px>

### We will run out of memory, like this:

```bash
torch.OutOfMemoryError: CUDA out of memory. Tried to allocate 5.38 GiB. GPU 0 has a total capacity of 23.55 GiB of which 5.32 GiB is free. Including non-PyTorch memory, this process has 0 bytes memory in use. Of the allocated memory 16.66 GiB is allocated by PyTorch, and 1.10 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
  0%|          | 0/600 [00:01<?, ?it/s]
```

## 2.2 In this case, we can use `quantization` to reduce the model size and memory usage.

- QLoRA: none -> 8,
- LoRA Configuration: Rank=8 -> 32, alpha(scaling coefficient)=16 -> 64,

<img src="./assets/QLoRA-Configuration.jpg" style="margin-left: 0px" width=1024px>


## 2.3 There was a dependency issue with `bitsandbytes`, Need to be set `0.44.1` version.
```bash
Traceback (most recent call last):
  File "/root/miniconda3/bin/llamafactory-cli", line 8, in <module>
    sys.exit(main())
  File "/root/autodl-tmp/LLaMA-Factory/src/llamafactory/cli.py", line 112, in main
    run_exp()
  File "/root/autodl-tmp/LLaMA-Factory/src/llamafactory/train/tuner.py", line 92, in run_exp
    _training_function(config={"args": args, "callbacks": callbacks})
  File "/root/autodl-tmp/LLaMA-Factory/src/llamafactory/train/tuner.py", line 66, in _training_function
    run_sft(model_args, data_args, training_args, finetuning_args, generating_args, callbacks)
  File "/root/autodl-tmp/LLaMA-Factory/src/llamafactory/train/sft/workflow.py", line 52, in run_sft
    model = load_model(tokenizer, model_args, finetuning_args, training_args.do_train)
  File "/root/autodl-tmp/LLaMA-Factory/src/llamafactory/model/loader.py", line 170, in load_model
    model = init_adapter(config, model, model_args, finetuning_args, is_trainable)
  File "/root/autodl-tmp/LLaMA-Factory/src/llamafactory/model/adapter.py", line 299, in init_adapter
    model = _setup_lora_tuning(
  File "/root/autodl-tmp/LLaMA-Factory/src/llamafactory/model/adapter.py", line 250, in _setup_lora_tuning
    model = get_peft_model(model, lora_config)
  File "/root/miniconda3/lib/python3.10/site-packages/peft/mapping.py", line 183, in get_peft_model
    return MODEL_TYPE_TO_PEFT_MODEL_MAPPING[peft_config.task_type](
  File "/root/miniconda3/lib/python3.10/site-packages/peft/peft_model.py", line 1542, in __init__
    super().__init__(model, peft_config, adapter_name, **kwargs)
  File "/root/miniconda3/lib/python3.10/site-packages/peft/peft_model.py", line 155, in __init__
    self.base_model = cls(model, {adapter_name: peft_config}, adapter_name)
  File "/root/miniconda3/lib/python3.10/site-packages/peft/tuners/lora/model.py", line 139, in __init__
    super().__init__(model, config, adapter_name)
  File "/root/miniconda3/lib/python3.10/site-packages/peft/tuners/tuners_utils.py", line 175, in __init__
    self.inject_adapter(self.model, adapter_name)
  File "/root/miniconda3/lib/python3.10/site-packages/peft/tuners/tuners_utils.py", line 431, in inject_adapter
    self._create_and_replace(peft_config, adapter_name, target, target_name, parent, current_key=key)
  File "/root/miniconda3/lib/python3.10/site-packages/peft/tuners/lora/model.py", line 224, in _create_and_replace
    new_module = self._create_new_module(lora_config, adapter_name, target, **kwargs)
  File "/root/miniconda3/lib/python3.10/site-packages/peft/tuners/lora/model.py", line 340, in _create_new_module
    new_module = dispatcher(target, adapter_name, lora_config=lora_config, **kwargs)
  File "/root/miniconda3/lib/python3.10/site-packages/peft/tuners/lora/bnb.py", line 273, in dispatch_bnb_8bit
    "memory_efficient_backward": target.state.memory_efficient_backward,
AttributeError: 'MatmulLtState' object has no attribute 'memory_efficient_backward'
```

In [ ]:
# !pip install bitsandbytes==0.44.1

## 2.4 Now, we can see trigger the QLoRA training process.

<img src="./assets/QLoRA-Training.jpg" style="margin-left: 0px" width=1024px>

## 2.5 `nvitop` command to monitor GPU usage:

<img src="./assets/nvitop.jpg" style="margin-left: 0px" width=1024px>

## 2.6 LLAMA-Factory logging:

```bash
root@autodl-container-fb0c4680e1-702ff0ba:~/autodl-tmp/LLaMA-Factory# llamafactory-cli webui
Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.
[WARNING|2025-01-19 06:25:38] llamafactory.hparams.parser:162 >> We recommend enable `upcast_layernorm` in quantized training.
[INFO|2025-01-19 06:25:38] llamafactory.hparams.parser:373 >> Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.bfloat16
[INFO|configuration_utils.py:677] 2025-01-19 06:25:38,566 >> loading configuration file /root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B/config.json
[INFO|configuration_utils.py:746] 2025-01-19 06:25:38,567 >> Model config LlamaConfig {
  "_name_or_path": "/root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "float32",
  "transformers_version": "4.46.1",
  "use_cache": true,
  "vocab_size": 128256
}

[INFO|tokenization_utils_base.py:2209] 2025-01-19 06:25:38,568 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:2209] 2025-01-19 06:25:38,568 >> loading file tokenizer.model
[INFO|tokenization_utils_base.py:2209] 2025-01-19 06:25:38,568 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2209] 2025-01-19 06:25:38,569 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2209] 2025-01-19 06:25:38,569 >> loading file tokenizer_config.json
[INFO|tokenization_utils_base.py:2475] 2025-01-19 06:25:38,934 >> Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
[INFO|configuration_utils.py:677] 2025-01-19 06:25:38,935 >> loading configuration file /root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B/config.json
[INFO|configuration_utils.py:746] 2025-01-19 06:25:38,936 >> Model config LlamaConfig {
  "_name_or_path": "/root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "float32",
  "transformers_version": "4.46.1",
  "use_cache": true,
  "vocab_size": 128256
}

[INFO|tokenization_utils_base.py:2209] 2025-01-19 06:25:38,937 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:2209] 2025-01-19 06:25:38,937 >> loading file tokenizer.model
[INFO|tokenization_utils_base.py:2209] 2025-01-19 06:25:38,937 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:2209] 2025-01-19 06:25:38,937 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:2209] 2025-01-19 06:25:38,937 >> loading file tokenizer_config.json
[INFO|tokenization_utils_base.py:2475] 2025-01-19 06:25:39,281 >> Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
[INFO|2025-01-19 06:25:39] llamafactory.data.template:157 >> Add <|eot_id|>,<|eom_id|> to stop words.
[INFO|2025-01-19 06:25:39] llamafactory.data.loader:157 >> Loading dataset ruozhiba.json...
training example:
input_ids:
[128000, 128006, 882, 128007, 271, 92780, 122679, 48044, 64209, 101171, 237, 35287, 98806, 27327, 76706, 103054, 11571, 128009, 128006, 78191, 128007, 271, 27327, 3922, 17792, 22656, 37507, 81258, 112986, 48044, 64209, 101171, 237, 1811, 128009]
inputs:
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

只剩一个心脏了还能活吗？<|eot_id|><|start_header_id|>assistant<|end_header_id|>

能，人本来就只有一个心脏。<|eot_id|>
label_ids:
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 27327, 3922, 17792, 22656, 37507, 81258, 112986, 48044, 64209, 101171, 237, 1811, 128009]
labels:
能，人本来就只有一个心脏。<|eot_id|>
[INFO|configuration_utils.py:677] 2025-01-19 06:25:40,237 >> loading configuration file /root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B/config.json
[INFO|configuration_utils.py:746] 2025-01-19 06:25:40,238 >> Model config LlamaConfig {
  "_name_or_path": "/root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "float32",
  "transformers_version": "4.46.1",
  "use_cache": true,
  "vocab_size": 128256
}

[INFO|2025-01-19 06:25:40] llamafactory.model.model_utils.quantization:157 >> Quantizing model to 8 bit with bitsandbytes.
[INFO|modeling_utils.py:3934] 2025-01-19 06:25:40,338 >> loading weights file /root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B/model.safetensors.index.json
[INFO|modeling_utils.py:1670] 2025-01-19 06:25:40,338 >> Instantiating LlamaForCausalLM model under default dtype torch.bfloat16.
[INFO|configuration_utils.py:1096] 2025-01-19 06:25:40,339 >> Generate config GenerationConfig {
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ]
}

Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.37it/s]
[INFO|modeling_utils.py:4800] 2025-01-19 06:25:41,879 >> All model checkpoint weights were used when initializing LlamaForCausalLM.

[INFO|modeling_utils.py:4808] 2025-01-19 06:25:41,879 >> All the weights of LlamaForCausalLM were initialized from the model checkpoint at /root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LlamaForCausalLM for predictions without further training.
[INFO|configuration_utils.py:1049] 2025-01-19 06:25:41,881 >> loading configuration file /root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B/generation_config.json
[INFO|configuration_utils.py:1096] 2025-01-19 06:25:41,881 >> Generate config GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "temperature": 0.6,
  "top_p": 0.9
}

[INFO|2025-01-19 06:25:41] llamafactory.model.model_utils.checkpointing:157 >> Gradient checkpointing enabled.
[INFO|2025-01-19 06:25:41] llamafactory.model.model_utils.attention:157 >> Using torch SDPA for faster training and inference.
[INFO|2025-01-19 06:25:41] llamafactory.model.adapter:157 >> Upcasting trainable params to float32.
[INFO|2025-01-19 06:25:41] llamafactory.model.adapter:157 >> Fine-tuning method: LoRA
[INFO|2025-01-19 06:25:41] llamafactory.model.model_utils.misc:157 >> Found linear modules: gate_proj,q_proj,down_proj,o_proj,v_proj,up_proj,k_proj
[INFO|2025-01-19 06:25:42] llamafactory.model.loader:157 >> trainable params: 22,544,384 || all params: 1,258,358,784 || trainable%: 1.7916
[INFO|trainer.py:698] 2025-01-19 06:25:42,206 >> Using auto half precision backend
[INFO|trainer.py:2313] 2025-01-19 06:25:42,435 >> ***** Running training *****
[INFO|trainer.py:2314] 2025-01-19 06:25:42,435 >>   Num examples = 1,496
[INFO|trainer.py:2315] 2025-01-19 06:25:42,435 >>   Num Epochs = 100
[INFO|trainer.py:2316] 2025-01-19 06:25:42,435 >>   Instantaneous batch size per device = 30
[INFO|trainer.py:2319] 2025-01-19 06:25:42,435 >>   Total train batch size (w. parallel, distributed & accumulation) = 240
[INFO|trainer.py:2320] 2025-01-19 06:25:42,435 >>   Gradient Accumulation steps = 8
[INFO|trainer.py:2321] 2025-01-19 06:25:42,435 >>   Total optimization steps = 600
[INFO|trainer.py:2322] 2025-01-19 06:25:42,437 >>   Number of trainable parameters = 22,544,384
  0%|                                                                                                                    | 0/600 [00:00<?, ?it/s]/root/miniconda3/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
  1%|▉                                                                                                         | 5/600 [00:32<1:02:46,  6.33s/it][INFO|2025-01-19 06:26:15] llamafactory.train.callbacks:157 >> {'loss': 1.7882, 'learning_rate': 4.9991e-05, 'epoch': 0.80}
{'loss': 1.7882, 'grad_norm': 0.5794951915740967, 'learning_rate': 4.999143312438893e-05, 'epoch': 0.8}
  2%|█▊                                                                                                       | 10/600 [01:04<1:02:30,  6.36s/it][INFO|2025-01-19 06:26:46] llamafactory.train.callbacks:157 >> {'loss': 1.9503, 'learning_rate': 4.9966e-05, 'epoch': 1.60}
{'loss': 1.9503, 'grad_norm': 0.4185701608657837, 'learning_rate': 4.996573836886435e-05, 'epoch': 1.6}
  2%|██▋                                                                                                      | 15/600 [01:38<1:06:23,  6.81s/it][INFO|2025-01-19 06:27:20] llamafactory.train.callbacks:157 >> {'loss': 1.8624, 'learning_rate': 4.9923e-05, 'epoch': 2.40}
{'loss': 1.8624, 'grad_norm': 0.4220849573612213, 'learning_rate': 4.99229333433282e-05, 'epoch': 2.4}
  3%|███▌                                                                                                       | 20/600 [02:08<59:52,  6.19s/it][INFO|2025-01-19 06:27:51] llamafactory.train.callbacks:157 >> {'loss': 1.8104, 'learning_rate': 4.9863e-05, 'epoch': 3.20}
{'loss': 1.8104, 'grad_norm': 0.35836485028266907, 'learning_rate': 4.9863047384206835e-05, 'epoch': 3.2}
  4%|████▍                                                                                                    | 25/600 [02:41<1:03:02,  6.58s/it][INFO|2025-01-19 06:28:24] llamafactory.train.callbacks:157 >> {'loss': 1.7765, 'learning_rate': 4.9786e-05, 'epoch': 4.00}
{'loss': 1.7765, 'grad_norm': 0.8218790292739868, 'learning_rate': 4.9786121534345265e-05, 'epoch': 4.0}
  5%|█████▎                                                                                                   | 30/600 [03:15<1:02:37,  6.59s/it][INFO|2025-01-19 06:28:57] llamafactory.train.callbacks:157 >> {'loss': 1.5137, 'learning_rate': 4.9692e-05, 'epoch': 4.80}
{'loss': 1.5137, 'grad_norm': 0.3808193504810333, 'learning_rate': 4.9692208514878444e-05, 'epoch': 4.8}
  6%|██████▏                                                                                                  | 35/600 [03:47<1:00:57,  6.47s/it][INFO|2025-01-19 06:29:29] llamafactory.train.callbacks:157 >> {'loss': 1.6818, 'learning_rate': 4.9581e-05, 'epoch': 5.60}
{'loss': 1.6818, 'grad_norm': 0.37492719292640686, 'learning_rate': 4.958137268909887e-05, 'epoch': 5.6}
  7%|███████                                                                                                  | 40/600 [04:18<1:00:34,  6.49s/it][INFO|2025-01-19 06:30:01] llamafactory.train.callbacks:157 >> {'loss': 1.6692, 'learning_rate': 4.9454e-05, 'epoch': 6.40}
{'loss': 1.6692, 'grad_norm': 0.3768139183521271, 'learning_rate': 4.9453690018345144e-05, 'epoch': 6.4}
  7%|███████▏                                                                                                 | 41/600 [04:25<1:00:06,  6.45s/it]
```

## 2.7 After 600 steps training, we can see the training process is finished.

<img src="./assets/QLoRA-Training-Completed.jpg" style="margin-left: 0px" width=1024px>

## 2.8 Export Model with merged training parameters: (Only QLoRA trained + all parameters merged)

<img src="./assets/QLoRA-Training-Output1.jpg" style="margin-left: 0px" width=1024px>

### Logging:

```bash
[INFO|modeling_utils.py:4800] 2025-01-19 07:33:56,855 >> All model checkpoint weights were used when initializing LlamaForCausalLM.

[INFO|modeling_utils.py:4808] 2025-01-19 07:33:56,855 >> All the weights of LlamaForCausalLM were initialized from the model checkpoint at /root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LlamaForCausalLM for predictions without further training.
[INFO|configuration_utils.py:1049] 2025-01-19 07:33:56,857 >> loading configuration file /root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B/generation_config.json
[INFO|configuration_utils.py:1096] 2025-01-19 07:33:56,857 >> Generate config GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "temperature": 0.6,
  "top_p": 0.9
}

[INFO|2025-01-19 07:33:56] llamafactory.model.model_utils.attention:157 >> Using torch SDPA for faster training and inference.
[INFO|2025-01-19 07:33:57] llamafactory.model.adapter:157 >> Merged 1 adapter(s).
[INFO|2025-01-19 07:33:57] llamafactory.model.adapter:157 >> Loaded adapter(s): saves/Llama-3.2-1B/lora/train_2025-01-19-06-24-25
[INFO|2025-01-19 07:33:57] llamafactory.model.loader:157 >> all params: 1,235,814,400
[INFO|2025-01-19 07:33:57] llamafactory.train.tuner:157 >> Convert model dtype to: torch.float32.
[INFO|configuration_utils.py:414] 2025-01-19 07:33:57,380 >> Configuration saved in /root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B-QLoRA/config.json
[INFO|configuration_utils.py:865] 2025-01-19 07:33:57,381 >> Configuration saved in /root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B-QLoRA/generation_config.json
[INFO|modeling_utils.py:3043] 2025-01-19 07:34:05,040 >> The model is bigger than the maximum size per checkpoint (3GB) and is going to be split in 2 checkpoint shards. You can find where each parameters has been saved in the index located at /root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B-QLoRA/model.safetensors.index.json.
[INFO|tokenization_utils_base.py:2646] 2025-01-19 07:34:05,043 >> tokenizer config file saved in /root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B-QLoRA/tokenizer_config.json
[INFO|tokenization_utils_base.py:2655] 2025-01-19 07:34:05,043 >> Special tokens file saved in /root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B-QLoRA/special_tokens_map.json
```

## 2.9 Export Model with quantization parameters: (QLoRA trained + quantized export)

<img src="./assets/QLoRA-Training-Output2.jpg" style="margin-left: 0px" width=1024px>

### Logging:

```bash
[INFO|modeling_utils.py:3934] 2025-01-19 07:36:34,187 >> loading weights file /root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B-QLoRA/model.safetensors.index.json
[INFO|modeling_utils.py:1670] 2025-01-19 07:36:34,187 >> Instantiating LlamaForCausalLM model under default dtype torch.float16.
[INFO|configuration_utils.py:1096] 2025-01-19 07:36:34,399 >> Generate config GenerationConfig {
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ]
}

Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.82it/s]
[INFO|modeling_utils.py:4800] 2025-01-19 07:36:35,532 >> All model checkpoint weights were used when initializing LlamaForCausalLM.

[INFO|modeling_utils.py:4808] 2025-01-19 07:36:35,532 >> All the weights of LlamaForCausalLM were initialized from the model checkpoint at /root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B-QLoRA.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LlamaForCausalLM for predictions without further training.
[INFO|configuration_utils.py:1049] 2025-01-19 07:36:35,534 >> loading configuration file /root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B-QLoRA/generation_config.json
[INFO|configuration_utils.py:1096] 2025-01-19 07:36:35,534 >> Generate config GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "temperature": 0.6,
  "top_p": 0.9
}

Quantizing model.layers blocks : 100%|███████████████████████████████████████████████████████████████████████████| 16/16 [01:39<00:00,  6.22s/it]
/root/miniconda3/lib/python3.10/site-packages/transformers/modeling_utils.py:5006: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
[WARNING|logging.py:328] 2025-01-19 07:38:15,264 >> `loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
[INFO|2025-01-19 07:38:54] llamafactory.model.model_utils.attention:157 >> Using torch SDPA for faster training and inference.
[INFO|2025-01-19 07:38:54] llamafactory.model.loader:157 >> all params: 262,735,872
[INFO|configuration_utils.py:414] 2025-01-19 07:38:54,432 >> Configuration saved in /root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B-QLoRA-Q8/config.json
[INFO|configuration_utils.py:865] 2025-01-19 07:38:54,433 >> Configuration saved in /root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B-QLoRA-Q8/generation_config.json
[INFO|modeling_utils.py:3035] 2025-01-19 07:38:56,364 >> Model weights saved in /root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B-QLoRA-Q8/model.safetensors
[INFO|tokenization_utils_base.py:2646] 2025-01-19 07:38:56,368 >> tokenizer config file saved in /root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B-QLoRA-Q8/tokenizer_config.json
[INFO|tokenization_utils_base.py:2655] 2025-01-19 07:38:56,368 >> Special tokens file saved in /root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B-QLoRA-Q8/special_tokens_map.json
```

## 3. Evaluate the fine-tuned model

### 3.1 Evaluate QLoRA with Chat:

<img src="./assets/QLoRA-Evaluation-Chat.jpg" style="margin-left: 0px" width=1024px>

### 3.2 Evaluate QLoRA with Valuation Dataset:

<img src="./assets/QLoRA-Evaluation-Predict.jpg" style="margin-left: 0px" width=1024px>

### 3.3 Evaluate QLoRA Quantized export model:

<img src="./assets/QLoRA-Evaluation-Quantized.jpg" style="margin-left: 0px" width=1024px>

### Logging:

```bash
[INFO|2025-01-19 18:52:49] logging.py:157 >> Loading 8-bit GPTQ-quantized model.

[INFO|2025-01-19 18:52:49] logging.py:157 >> Using KV cache for faster generation.

[INFO|2025-01-19 18:52:49] modeling_utils.py:3934 >> loading weights file /root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B-QLoRA-Q8/model.safetensors

[INFO|2025-01-19 18:52:49] modeling_utils.py:1670 >> Instantiating LlamaForCausalLM model under default dtype torch.float16.

[INFO|2025-01-19 18:52:49] configuration_utils.py:1096 >> Generate config GenerationConfig { "bos_token_id": 128000, "eos_token_id": [ 128001, 128008, 128009 ] }

[WARNING|2025-01-19 18:52:49] logging.py:328 >> loss_type=None was set in the config but it is unrecognised.Using the default loss: ForCausalLMLoss.

[INFO|2025-01-19 18:52:50] modeling_utils.py:4800 >> All model checkpoint weights were used when initializing LlamaForCausalLM.

[INFO|2025-01-19 18:52:50] modeling_utils.py:4808 >> All the weights of LlamaForCausalLM were initialized from the model checkpoint at /root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B-QLoRA-Q8. If your task is similar to the task the model of the checkpoint was trained on, you can already use LlamaForCausalLM for predictions without further training.

[INFO|2025-01-19 18:52:50] configuration_utils.py:1049 >> loading configuration file /root/autodl-tmp/models/UnicomAI/Unichat-llama3.2-Chinese-1B-QLoRA-Q8/generation_config.json

[INFO|2025-01-19 18:52:50] configuration_utils.py:1096 >> Generate config GenerationConfig { "bos_token_id": 128000, "do_sample": true, "eos_token_id": [ 128001, 128008, 128009 ], "temperature": 0.6, "top_p": 0.9 }

[INFO|2025-01-19 18:52:50] logging.py:157 >> Using torch SDPA for faster training and inference.

[INFO|2025-01-19 18:52:50] logging.py:157 >> all params: 262,735,872

[WARNING|2025-01-19 18:52:50] logging.py:168 >> Batch generation can be very slow. Consider using scripts/vllm_infer.py instead.

[INFO|2025-01-19 18:52:50] trainer.py:4117 >> ***** Running Prediction *****

[INFO|2025-01-19 18:52:50] trainer.py:4119 >> Num examples = 8

[INFO|2025-01-19 18:52:50] trainer.py:4122 >> Batch size = 2
```